# LangChainを用いて、昨日や一週間前のことを会話する

もしマイ・チャットボットがいたら、当然、数日前の会話内容を覚えていて欲しいですよね？  
以下では、過去の会話内容に基づいた会話をするチャットボットを、LangChainを用いて作っていきます。

1. 環境構築
2. 会話内容の保存・読込
3. シンプルな方式（過去の会話を、文脈情報として与える）
4. Agentを用いる方式（過去の会話を、ChatBot自身で呼び出す）


## 1. 環境構築

In [1]:
! pip install openai langchain

In [2]:
import os
os.environ['OPENAI_API_KEY'] = 'PLEASE CHANGE HERE'

## 2. 会話内容の保存・読込

### LangChainでの会話内容の保存
- LangChainでは、会話は`Memory`の内部に、`Message`のリストとして保存されます
- 会話内容を`dict`型に変換する方法は、公式ページで紹介されています
    - https://python.langchain.com/docs/modules/memory/#saving-message-history

In [3]:
from pprint import pprint
from langchain.memory import ConversationBufferMemory
from langchain.schema import messages_from_dict, messages_to_dict

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi!"}, {"output": "whats up?"})

chat_messages = memory.load_memory_variables({})['history']
history_dict = messages_to_dict(chat_messages)
pprint(history_dict)

[{'data': {'additional_kwargs': {}, 'content': 'hi!', 'example': False},
  'type': 'human'},
 {'data': {'additional_kwargs': {}, 'content': 'whats up?', 'example': False},
  'type': 'ai'}]


### 保存・読込用の関数

In [4]:
# 会話内容の保存・読込の関数
import json

def save_conversation_from_memory(json_filepath, memory, memory_key='history', **kwargs):
    history_messages = memory.load_memory_variables({})[memory_key]
    history_dict = messages_to_dict(history_messages)
    dat = dict({memory_key: history_dict}, **kwargs)  # 任意の情報を保存可能

    with open(json_filepath, 'w') as fout:
        json.dump(dat, fout)
    print('Saved ...', json_filepath)

def load_conversation_from_json(json_filepath, memory_key='history'):
    with open(json_filepath, 'r') as fin:
        dict_all = json.load(fin)
        ret_dict = {}
        for k, v in dict_all.items():
            if k == memory_key:
                ret_dict[k] = messages_from_dict(v)
            else:
                ret_dict[k] = v
    return ret_dict

json_file = 'test_conversation.json'
save_conversation_from_memory(json_file, memory)
load_conversation_from_json(json_file)

Saved ... test_conversation.json


{'history': [HumanMessage(content='hi!', additional_kwargs={}, example=False),
  AIMessage(content='whats up?', additional_kwargs={}, example=False)]}

### 実験用に、会話内容を保存する

In [5]:
# json_file, datetime, chat_history
past_conversations = [
    [
        'chat_history_20230628.json',
        '2023-06-28 19:03:42',
        [
            ({"input": "最近暑くなってきたからさっぱりしたモノが食べたくて、冷麺を食べたよ。"},
             {"output": "いいね！ちなみに、盛岡冷麺、韓国冷麺、どっち…？"}),
        ]
    ],
    [
        'chat_history_20230703.json',
        '2023-07-03 21:53:02',
        [
            ({"input": "今日は友達とイタリアンに行ったよ。イカスミパスタと真鯛のカルパッチョを食べたんだけど、どっちも美味しかったぁ。"},
             {"output": "いいなぁ。僕もイタリアン食べたい！！"}),
        ]
    ],
    [
        'chat_history_20230704.json',
        '2023-07-04 20:11:56',
        [
            ({"input": "今日の昼は中華レストランでよだれ鶏を食べんたんだ。最近暑いからか、あの酸味と辛味を欲してしまうんだよね"},
             {"output": "暑いと酸味と辛味を欲するの？なんで？"}),
        ]
    ],
]

# 日別で会話内容を保存
for json_file, datetime_str, chat_history in past_conversations:
    memory = ConversationBufferMemory(return_messages=True)
    for input_output_tuple in chat_history:
        memory.save_context(*input_output_tuple)
    # datetimeも一緒に保存
    save_conversation_from_memory(json_file, memory, datetime=datetime_str)


Saved ... chat_history_20230628.json
Saved ... chat_history_20230703.json
Saved ... chat_history_20230704.json


## 3. シンプルな方式
過去の会話を、文脈情報として与える

In [6]:
# 過去の会話を読み込む
import glob
from langchain.schema import get_buffer_string  # Messageのリストを文字列に変換する関数


def generate_context_from_past_chats(chat_history_files):
    past_chats_text = ''
    for history_file in chat_history_files:
        d = load_conversation_from_json(history_file)
        dt = d['datetime']
        msgs = d['history']
        msgs_str = get_buffer_string(msgs)  #, ai_prefix='あなた', human_prefix='僕')
        past_chats_text += '【日時】 {:s}\n{:s}\n---\n'.format(dt, msgs_str)

    past_chat_prompt = (
        '以下は、過去の会話です。\n'
        '================\n'
        '{:s}\n'
        '================\n\n'
    ).format(past_chats_text.rstrip('---\n'))

    return past_chat_prompt


past_chats_files = glob.glob('chat_history_*.json')
past_chats_files.sort()
past_chats_context = generate_context_from_past_chats(past_chats_files)
print(past_chats_context)

以下は、過去の会話です。
【日時】 2023-06-28 19:03:42
Human: 最近暑くなってきたからさっぱりしたモノが食べたくて、冷麺を食べたよ。
AI: いいね！ちなみに、盛岡冷麺、韓国冷麺、どっち…？
---
【日時】 2023-07-03 21:53:02
Human: 今日は友達とイタリアンに行ったよ。イカスミパスタと真鯛のカルパッチョを食べたんだけど、どっちも美味しかったぁ。
AI: いいなぁ。僕もイタリアン食べたい！！
---
【日時】 2023-07-04 20:11:56
Human: 今日の昼は中華レストランでよだれ鶏を食べんたんだ。最近暑いからか、あの酸味と辛味を欲してしまうんだよね
AI: 暑いと酸味と辛味を欲するの？なんで？




In [7]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory


dt_now = '2023-07-05 19:17:28'

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "あなたはエージェント型チャットボットです。\n"
        "過去の会話を参照しながら対話者（僕）と会話することができます。\n"
        "発言は、100字以内で短く返してください。\n\n"
        f"{past_chats_context}"
        f"現在の日時：{dt_now}\n\nそれでは、会話開始です。"
    ),
    MessagesPlaceholder(variable_name="today_history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

def create_new_chatbot(temperature=0, verbose=True):
    llm = ChatOpenAI(temperature=temperature, model_name="gpt-3.5-turbo")
    memory = ConversationBufferMemory(return_messages=True, memory_key='today_history')
    chatbot = LLMChain(memory=memory, prompt=prompt, llm=llm, verbose=verbose)
    return chatbot

In [8]:
chatbot = create_new_chatbot(verbose=True)
chatbot.predict(input="一昨日僕が食べた料理を答えてみて。")



> Entering new  chain...
Prompt after formatting:
System: あなたはエージェント型チャットボットです。
過去の会話を参照しながら対話者（僕）と会話することができます。
発言は、100字以内で短く返してください。

以下は、過去の会話です。
【日時】 2023-06-28 19:03:42
Human: 最近暑くなってきたからさっぱりしたモノが食べたくて、冷麺を食べたよ。
AI: いいね！ちなみに、盛岡冷麺、韓国冷麺、どっち…？
---
【日時】 2023-07-03 21:53:02
Human: 今日は友達とイタリアンに行ったよ。イカスミパスタと真鯛のカルパッチョを食べたんだけど、どっちも美味しかったぁ。
AI: いいなぁ。僕もイタリアン食べたい！！
---
【日時】 2023-07-04 20:11:56
Human: 今日の昼は中華レストランでよだれ鶏を食べんたんだ。最近暑いからか、あの酸味と辛味を欲してしまうんだよね
AI: 暑いと酸味と辛味を欲するの？なんで？

現在の日時：2023-07-05 19:17:28

それでは、会話開始です。
Human: 一昨日僕が食べた料理を答えてみて。

> Finished chain.


'一昨日はイタリアンのイカスミパスタと真鯛のカルパッチョを食べたよね。'

In [9]:
chatbot = create_new_chatbot(verbose=False)
chatbot.predict(input="7/3に僕が食べた料理を答えてみて。")

'イカスミパスタと真鯛のカルパッチョを食べたんだよね。'

In [10]:
# 失敗例
chatbot = create_new_chatbot(verbose=False)
chatbot.predict(input="昨日食べた料理が美味しかったから、自分でも作ってみたよ。何を作ったでしょうか?")

'それは素晴らしいですね！何を作ったのか教えてください。'

In [11]:
# 失敗例
chatbot = create_new_chatbot(verbose=False)
chatbot.predict(input="昨日食べた料理は何でしょうか?")

'すみません、私は過去の会話しか覚えていませんので、昨日の料理についてはわかりません。何か他の話題でお話ししましょうか？'

In [12]:
chatbot = create_new_chatbot(verbose=False)
chatbot.predict(input="7/4に食べた料理は何でしょうか？")

'7/4に食べた料理は「よだれ鶏」です。'

In [27]:
chatbot = create_new_chatbot(verbose=False)
chatbot.predict(input="僕、一週間前って何食べたっけ？")

'一週間前は、冷麺を食べたようですね。暑くなってきたからさっぱりしたモノが食べたくて、冷麺を選んだんですよ。'

## 4. Agentを用いる方法
過去の会話を、ChatBot自身で呼び出す

In [14]:
from langchain import OpenAI

llm = OpenAI(temperature=0)  # text-davinci-003

In [15]:
from langchain.tools import tool

# 「昨日」「明日」といった単語を、日付に変更するツール
@tool("get_date_from_keyword")
def get_date_from_keyword(date_keyward: str) -> str:
    """Get specific date in 'yyyy/mm/dd' format from Japanese keyword (e.g. '昨日', '明後日')."""
    dt_today = '2023/07/05'  # Currently hard coded.
    template1 = 'Translate japanese word "{}" into English.'
    template2 = f'Today is {dt_today}. Tell the date the word "{{}}" means in "2023/01/12" format.'

    word = llm(template1.format(date_keyward)).lstrip('\n')  # 昨日 --> yesterday
    dt_target = llm(template2.format(word)).lstrip('\n')      # yesterday --> 2023/07/04
    print(date_keyward, word, dt_target)
    return dt_target

print(get_date_from_keyword('おととい'))

おととい Two days ago 2023/07/03
2023/07/03


In [16]:
import re
from langchain.tools import Tool


class RememberPastTool():

    def __init__(self, chat_history_dir='./', summarize=False):
        self._data_dir = chat_history_dir
        self._summarize = summarize

    def get_chat_history_text(self, date):
        history_file = os.path.join(self._data_dir, f"chat_history_{date.replace('/', '')}.json")
        if not os.path.isfile(history_file):
            return 'その日は会話していないようです'
        d = load_conversation_from_json(history_file)
        msgs_str = get_buffer_string(d['history'])
        return msgs_str

    def remember_past(self, query: str) -> str:
        """Get stored past chat history. It is useful to talk about the past experience.
        Query must be date (e.g. 2023/07/05)"""
        query_org = query
        if re.match(r'\d{4}/\d{2}/\d{2}', query) is None:  # is not date format
            query = get_date_from_keyword(query)
        text = self.get_chat_history_text(query)
        #---------------------
        if self._summarize:
            char_num = len(text) // 2
            text = llm(f'以下の会話を{char_num}字以内で要約して: {text}\n').lstrip()
        #---------------------
        return (f'以下は、{query_org}の会話です。\n---\n{text}\n---\n'
                'Please recall "New input". Then, answer to "New input" based on this information.\n')
                # 'まず、"New input"を思い出してください。そして、この情報を参照して"New input"に回答してください。')

remember_past_tool = Tool.from_function(
    func=RememberPastTool().remember_past,
    name="recall_past_chats",
    description=("MUST use this when talking about past experience. You can recall past chat conversations. "
                 "Input can be date (yyyy/mm/dd) or word (e.g. '昨日', '一週間前').")
)

print(remember_past_tool('2023/07/03'))

以下は、2023/07/03の会話です。
---
Human: 今日は友達とイタリアンに行ったよ。イカスミパスタと真鯛のカルパッチョを食べたんだけど、どっちも美味しかったぁ。
AI: いいなぁ。僕もイタリアン食べたい！！
---
Please recall "New input". Then, answer to "New input" based on this information.



In [17]:
print(remember_past_tool('一週間前'))

一週間前 One week ago 2023/06/28
以下は、一週間前の会話です。
---
Human: 最近暑くなってきたからさっぱりしたモノが食べたくて、冷麺を食べたよ。
AI: いいね！ちなみに、盛岡冷麺、韓国冷麺、どっち…？
---
Please recall "New input". Then, answer to "New input" based on this information.



In [18]:
from langchain import OpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory


def create_new_agent(temparature=0, verbose=True, with_initialize=False):
    llm_chat = OpenAI(temperature=temparature)
    memory = ConversationBufferMemory(memory_key="chat_history")

    tools = [remember_past_tool, get_date_from_keyword]

    agent_chain = initialize_agent(
        tools, llm_chat,
        agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=verbose,
        max_execution_time=3,   # https://python.langchain.com/docs/modules/agents/how_to/max_time_limit
        early_stopping_method="generate",
        )

    # 初期化（あまり効果はなかった）
    if with_initialize:
        agent_chain.run(input=(
            "以下はシステム入力です\n"
            "-------------\n"
            "あなたは過去の会話内容に基づいて会話をするエージェント型チャットボットです。\n"
            "以下を 【必ず】 守ってください！！\n"
            "- ユーザの発言に回答する\n"
            "- 情報が不足している場合は、正直にわからないと答える。絶対に、勝手に話を作り出さない\n"
            "- 50字以内で答える\n"
            "-------------\n"
            ))

    # # 内部のLLMChainの入力を見る（verbose=True）には、以下のようにする
    # # cf. https://github.com/hwchase17/langchain/issues/912#issuecomment-1426646112
    # agent_chain.agent.llm_chain.verbose=True

    # agent_chain.agent.llm_chain.prompt.template = \
    #     agent_chain.agent.llm_chain.prompt.template.replace(
    #         "New input: {input}",
    #         "New input: {input}\n（ユーザの発言です。必ずこの発言に回答してください)",
    #     )

    return agent_chain

In [19]:
agent_chain = create_new_agent()
agent_chain.run(input="こんにちは")



> Entering new  chain...

Thought: Do I need to use a tool? No
AI: こんにちは！どうかしましたか？

> Finished chain.


'こんにちは！どうかしましたか？'

In [20]:
agent_chain.run(input="昨日は何月何日でしょうか？")



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: get_date_from_keyword
Action Input: 昨日昨日 Yesterday 2023/07/04

Observation: 2023/07/04
Thought: Do I need to use a tool? No
AI: 昨日は2023年7月4日です。

> Finished chain.


'昨日は2023年7月4日です。'

In [21]:
agent_chain = create_new_agent()
agent_chain.run(input="一昨日、僕が何を食べたか、答えてください！")



> Entering new  chain...

Thought: Do I need to use a tool? Yes
Action: recall_past_chats
Action Input: 一昨日一昨日 Two days ago 2023/07/03

Observation: 以下は、一昨日の会話です。
---
Human: 今日は友達とイタリアンに行ったよ。イカスミパスタと真鯛のカルパッチョを食べたんだけど、どっちも美味しかったぁ。
AI: いいなぁ。僕もイタリアン食べたい！！
---
Please recall "New input". Then, answer to "New input" based on this information.

Thought: Do I need to use a tool? No
AI: 一昨日は友達とイタリアンに行って、イカスミパスタと真鯛のカルパッチョを食べたそうですね！

> Finished chain.


'一昨日は友達とイタリアンに行って、イカスミパスタと真鯛のカルパッチョを食べたそうですね！'

In [22]:
# 若干微妙
agent_chain = create_new_agent()
agent_chain.run(input="僕、一週間前って何食べたっけ？")



> Entering new  chain...

Thought: Do I need to use a tool? Yes
Action: recall_past_chats
Action Input: one week agoone week ago One Week Ago 2023/06/28

Observation: 以下は、one week agoの会話です。
---
Human: 最近暑くなってきたからさっぱりしたモノが食べたくて、冷麺を食べたよ。
AI: いいね！ちなみに、盛岡冷麺、韓国冷麺、どっち…？
---
Please recall "New input". Then, answer to "New input" based on this information.

Thought: Do I need to use a tool? No
AI: 冷麺なら、盛岡冷麺と韓国冷麺がありますね！どちらを食べましたか？

> Finished chain.


'冷麺なら、盛岡冷麺と韓国冷麺がありますね！どちらを食べましたか？'

In [23]:
# 若干微妙
agent_chain = create_new_agent()
agent_chain.run(input="昨日僕が食べた料理を答えてみて。")



> Entering new  chain...

Thought: Do I need to use a tool? Yes
Action: recall_past_chats
Action Input: 昨日昨日 Yesterday 2023/07/04

Observation: 以下は、昨日の会話です。
---
Human: 今日の昼は中華レストランでよだれ鶏を食べんたんだ。最近暑いからか、あの酸味と辛味を欲してしまうんだよね
AI: 暑いと酸味と辛味を欲するの？なんで？
---
Please recall "New input". Then, answer to "New input" based on this information.

Thought: Do I need to use a tool? No
AI: 昨日は中華レストランでよだれ鶏を食べましたね。暑いからか、あの酸味と辛味がとても美味しかったです。

> Finished chain.


'昨日は中華レストランでよだれ鶏を食べましたね。暑いからか、あの酸味と辛味がとても美味しかったです。'

In [26]:
# 失敗例
agent_chain = create_new_agent()
agent_chain.run(input="一昨日の日付は？その日、僕が食べた料理は何？")



> Entering new  chain...

Thought: Do I need to use a tool? Yes
Action: get_date_from_keyword
Action Input: 一昨日一昨日 Two days ago 2023/07/03

Observation: 2023/07/03
Thought: Do I need to use a tool? No
AI: 一昨日は2023年7月3日ですね。その日は、あなたが食べた料理は何でしたか？

> Finished chain.


'一昨日は2023年7月3日ですね。その日は、あなたが食べた料理は何でしたか？'

In [25]:
print(agent_chain.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 